In [ ]:
from classy import Class
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

----
## 1. Obtain T(k)
----

In [ ]:
#redshift
z = 0
A_s = 2.100549e-09
n_s = 0.965

In [ ]:
LambdaCDM = Class()
LambdaCDM.set({'omega_b':0.0223828,'omega_cdm':0.1201075,'h':0.67810,'A_s':A_s,'n_s':n_s,'tau_reio':0.05430842,'N_ncdm':1, 'm_ncdm':0.06})
LambdaCDM.set({'output':'tCl,pCl,lCl,mPk,mTk','lensing':'yes','P_k_max_1/Mpc':3.0})
# lancio la classe
LambdaCDM.compute()


In [ ]:
one_time = LambdaCDM.get_transfer(z)
ka = one_time['k (h/Mpc)']
d_g = one_time['d_g']
d_b = one_time['d_b']
d_cdm = one_time['d_cdm']
d_ncdm = abs(one_time['d_ncdm[0]'])
d_m = one_time['d_m']
d_tot = one_time['d_tot']
phi = one_time['phi']
psi = one_time['psi']

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel(r'$k \,\,\,\, [h/\mathrm{Mpc}]$')
ax.set_ylabel(r'$T(k)$')
ax.set_title('Transfer Function $T(k)$')
#ax.plot(k, d_g, color='#424B54', lw=1, label='d_g')
#ax.plot(k, d_b, color='green', lw=1, label='d_b')
#ax.plot(k, d_cdm, color='b', lw=1, label='d_cdm')
#ax.plot(k, d_ncdm, color='k', lw=1, label='d_ncdm')
#ax.plot(k, d_m, color='red', lw=1, label='d_m')
#ax.plot(k, d_tot, color='purple', lw=1, label='d_tot')
ax.plot(ka, phi, color='violet', lw=1, label='psi')
ax.plot(ka, psi, color='cyan', lw=1, label='psi')
plt.show()

In [ ]:
primordial = LambdaCDM.get_primordial()

In [ ]:
k_piv = 0.05 
k_1 = primordial['k [1/Mpc]']
Ppk = []
for i in range(len(k_1)):
    Ppk.append(A_s*(k_1[i] / k_piv)**(LambdaCDM.n_s()-1))
Ppk_cl = primordial['P_scalar(k)']

$$
P_\mathrm{prim} = A_s \left(\frac{k}{k_\mathrm{pivot}}\right)^{n_s-1}
$$

in class P_primordiale è quello moltiplicato per k^3/2pi^2

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(12,4))
ax1.set_xscale('log')
ax1.set_yscale('log')
ax2.set_xscale('log')
ax2.set_yscale('log')
ax1.set_xlabel(r'$k \,\,\,\, [h/\mathrm{Mpc}]$')
ax1.set_ylabel(r'$P_0(k) \,\,\,\, [(h/\mathrm{Mpc})^3]$')
ax2.set_xlabel(r'$k \,\,\,\, [h/\mathrm{Mpc}]$')
ax2.set_ylabel(r'$P_0(k) \,\,\,\, [(h/\mathrm{Mpc})^3]$')
ax1.set_title('Primordial Power Matter Spectrum from CLASS')
ax2.set_title('Calculated Primordial Power Matter Spectrum')
ax1.plot(k_1, Ppk_cl)
ax2.plot(k_1, Ppk)


----
## MPS
----

In [ ]:
Pk = []
for k in k_1:
        Pk.append(LambdaCDM.pk(k * LambdaCDM.h(), 0.) * LambdaCDM.h() ** 3)

----
## Calcolo T_f
----

In [ ]:
a_f = 1/1+z
H_0 = LambdaCDM.h()*100
omega_mz = LambdaCDM.Omega_m()
c = 299792.458

In [ ]:
Phi_1 = [] #calcolato con il valore di P_prim di class
Phi_2 = [] #calcolato con il valore di P_prim calcolato da me
for i in range (len(k_1)):
    Phi_1.append(k_1[i]**(1.5)*np.sqrt(Pk[i]/Ppk_cl[i])*(3*(H_0**2)*omega_mz)/(2*((c*k_1[i])**2)*a_f))
    Phi_2.append(k_1[i]**(1.5)*np.sqrt(Pk[i]/Ppk[i])*(3*(H_0**2)*omega_mz)/(2*((c*k_1[i])**2)*a_f))


In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(12,4))
ax1.set_xscale('log')
ax1.set_yscale('log')
ax2.set_xscale('log')
ax2.set_yscale('log')
ax1.set_xlabel(r'$k \,\,\,\, [h/\mathrm{Mpc}]$')
ax1.set_ylabel(r'$T(k)$')
ax2.set_xlabel(r'$k \,\,\,\, [h/\mathrm{Mpc}]$')
ax2.set_ylabel(r'$T(k)$')
ax1.set_title('From CLASS')
ax2.set_title('Calcoli Miei')
ax1.plot(k_1, Phi_1)
ax2.plot(k_1, Phi_2)

In [ ]:
plt.loglog(k_1, Phi_2, label='mio')
plt.loglog(ka,phi, label='class')
plt.legend()


In [ ]:
import os
notebook_folder = os.path.dirname('')
data_folder = os.path.join(notebook_folder, "..", "Data")
def data_path(dat_id):
    return os.path.join(data_folder, dat_id)
def save_file(dat_id,df):
    df.to_csv(data_path(dat_id), index=False)

In [ ]:
data = {'k (h/Mpc)': ka, 'T(k)': phi}
df = pd.DataFrame(data)
save_file('TF_class',df)

In [ ]:
data = {'k (1/Mpc)': k_1, 'T(k)': Phi_2}
df = pd.DataFrame(data)
save_file('TF_custom',df)